# Washington PPE Forecasting
## Editable View
This document provides direct access to the underlying data informing the projections. In addition to using sliders to adjust stockpile days, you can edit burn rate assumptions and rerun the model on the spot.

We provide detailed annotations of how to adjust model parameters for readers that might not have experience in programming or the Python programming language. However, if you do have a programming background, we expose our source code here for even more interactivity and editability. Simply use the button below to toggle on/off the raw code.

This is an alpha version of this view, so there are a couple extra steps to get started:

1. In the menu above, click on the `Kernel` button. A drop-down menu will appear; click `Restart and Run All`.
2. This will trigger a prompt asking you if you want to restart the kernel and re-run the whole notebook. Click the red button, `Restart and Run All Cells`. If errors begin to occur at any point, repeat this step of `Restart and Run All Cells`.

At this point, setup is complete.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import os
import sys
import qgrid
import pathlib
import restart
import numpy as np
import pandas as pd
from ipywidgets import widgets
from restart import RestartModel
from restart import Data
from restart_datasets import data
from IPython.display import Javascript, display
from restart import util
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
from restart_datasets import data
import ipywidgets as widgets
import bqplot
from bqplot import pyplot as plt

qgrid.set_grid_option('forceFitColumns', False)
PATH = pathlib.Path(restart.__file__).parent.absolute()

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, \
                        IPython.notebook.ncells())'))
    
def create_run_button(description):
    button = widgets.Button(description=description)
    button.on_click(run_all)
    return button

gr_restart = RestartModel(
    config='wa_groups',
    population='oes',
    state='Washington',
    subpop='wa_groupings'
)

gr_model = gr_restart.model
gr_model.inventory.set_average_orders_per_period(gr_model.demand.demand_by_popsum1_total_rp1n_tc)

wa_burn_sheet = format_population(to_sheet(gr_model.demand.demand_per_unit_map_dn_um.df))
gr_pop = format_population(to_sheet(gr_model.population.population_pP_tr.df))
    

burn_df = gr_model.demand.demand_per_unit_map_dn_um.df
burn_df.index.name = "Population"

burn_rates = qgrid.show_grid(burn_df)

# Washington Burn Rates
Click on any of the cells below to adjust burn rate assumptions. When you're done, hit `Enter` and click the `Run Model` button and the updated model will be generated below (you'll have to scroll down a bit to see it). 

In [3]:
display(create_run_button("Run Model"))
widgets.VBox([burn_rates])

Button(description='Run Model', style=ButtonStyle())

## Burn Rate Visualization

In [4]:
# get updated burn rates
gr_model.demand.adjust_burn(burn_rates.get_changed_df().to_numpy())

# charting 
burn_chart = util.generate_group_bar_legend(gr_model.demand.demand_per_unit_map_dn_um.df)
widgets.VBox([burn_chart])

## Selected First Responders

In [5]:
def gr_dashboard(backstop):
    gr_set_stock(backstop)
    
def gr_display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['EMTs', 'Firefighters', 'Other Public Safety']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = util.generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def gr_set_stock(backstop):
    backstop = [backstop]
    gr_model.inventory.order(gr_model.inventory.inv_by_popsum1_total_rp1n_tc)
    gr_model.inventory.set_min_in_periods(backstop)
    gr_display_stock(gr_model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
gr_slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)
gr_out = widgets.interactive_output(gr_dashboard, {'backstop': gr_slider})
gr_sum_sheet = format_population(to_sheet(gr_model.population.pop_to_popsum1_per_unit_map_pp1_us.df))

widgets.VBox([burn_chart, gr_slider, gr_out])

## Selected Healthcare Populations

In [6]:
restart = RestartModel(config='des', population='oes', state='Washington', subpop='wa_tier2_opt2')
model = restart.model
model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = util.generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)

pop = format_population(to_sheet(model.population.population_pP_tr.df))

burn_chart = util.generate_group_bar_legend(model.demand.demand_per_unit_map_dn_um.df)
out = widgets.interactive_output(dashboard, {'backstop': slider})

pop = format_population(to_sheet(model.population.population_pP_tr.df))

widgets.VBox([slider, out])